In [14]:
import fitz  # PyMuPDF
from PIL import Image
import io

def pdf_to_images(file_object, max_pages=5, dpi=150):
    # Open the PDF file using PyMuPDF with a file-like object
    pdf_document = fitz.open(stream=file_object.read(), filetype="pdf")
    images = []
    
    # Loop through each page up to the maximum specified
    for page_number in range(min(max_pages, pdf_document.page_count)):
        page = pdf_document.load_page(page_number)   # Load page
        pix = page.get_pixmap(dpi=dpi)               # Render page to image with specified DPI
        
        # Convert the Pixmap to a PIL image
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        images.append(img)
        
    pdf_document.close()
    return images

In [ ]:
import pymupdf

doc = pymupdf.open("attention.pdf") # open a document

for page_index in range(len(doc)): # iterate over pdf pages
    page = doc[page_index] # get the page
    image_list = page.get_images()

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index}")
    else:
        print("No images found on page", page_index)

    for image_index, img in enumerate(image_list, start=1): # enumerate the image list
        xref = img[0] # get the XREF of the image
        pix = pymupdf.Pixmap(doc, xref) # create a Pixmap

        if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
            pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

        pix.save("page_%s-image_%s.png" % (page_index, image_index)) # save the image as png
        pix = None